In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

text = "A dog is playing with a ball"
tokens = tokenizer(text)

print(tokens["input_ids"])

/Users/abdulvajid/AI/PROJECTS/Image-Caption-Generation-VIT-From-Scratch/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/abdulvajid/AI/PROJECTS/Image-Caption-Generation-VIT-From-Scratch/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[101, 1037, 3899, 2003, 2652, 2007, 1037, 3608, 102]


In [2]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [3]:
import json

In [8]:
with open('archive/annotations_trainval2014/annotations/captions_train2014.json', 'r') as f:
    data = json.load(f)

In [19]:
import pandas as pd

In [24]:
images_data = pd.DataFrame(data['images'])
annotation_data = pd.DataFrame(data['annotations'])

In [41]:
data = annotation_data.merge(images_data, how='left', left_on='image_id', right_on='id')

In [45]:
data = data.drop(columns=['id_x', 'license', 'height', 'width', 'date_captured', 'flickr_url','coco_url', 'id_y', 'image_id'])

In [64]:
data.to_csv('processed_caption.csv', index=False)

In [63]:
import os
data['file_name'] = data['file_name'].apply(lambda x: os.path.join('archive/train2014/train2014', x))

In [59]:
def tokenize(sample):
    # Tokenize the caption column
    tokens = tokenizer(sample['caption'], padding='max_length', truncation=True, max_length=128)
    return tokens['input_ids']

# Apply tokenization to the caption column
data['caption_tokens'] = data['caption'].apply(lambda x: tokenize({'caption': x}))
print("Tokenization complete!")
print(data.head())

Tokenization complete!
                                             caption  \
0     A very clean and well decorated empty bathroom   
1  A panoramic view of a kitchen and all of its a...   
2  A blue and white bathroom with butterfly theme...   
3     A panoramic photo of a kitchen and dining room   
4  A graffiti-ed stop sign across the street from...   

                         file_name  \
0  COCO_train2014_000000318556.jpg   
1  COCO_train2014_000000116100.jpg   
2  COCO_train2014_000000318556.jpg   
3  COCO_train2014_000000116100.jpg   
4  COCO_train2014_000000379340.jpg   

                                      caption_tokens  
0  [101, 1037, 2200, 4550, 1998, 2092, 7429, 4064...  
1  [101, 1037, 6090, 6525, 7712, 3193, 1997, 1037...  
2  [101, 1037, 2630, 1998, 2317, 5723, 2007, 9112...  
3  [101, 1037, 6090, 6525, 7712, 6302, 1997, 1037...  
4  [101, 1037, 17990, 1011, 3968, 2644, 3696, 240...  


In [68]:
data.iloc[0]['caption']

'A very clean and well decorated empty bathroom'

In [69]:
from torch.utils.data import DataLoader, Dataset

class ImageCaptionData(Dataset):
    def __init__(self, path='processed_caption.csv'):
        super().__init__()
        self.path = path
        self.data = pd.read_csv(path)
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        return self.data.iloc[index]['file_name'], self.data.iloc[index]['caption_tokens']

In [70]:
dataset = ImageCaptionData()

In [72]:
dataset[0]

('archive/train2014/train2014/COCO_train2014_000000318556.jpg',
 '[101, 1037, 2200, 4550, 1998, 2092, 7429, 4064, 5723, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]')

In [76]:
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from model_args import Arguments
from torchvision import transforms
from PIL import Image

args = Arguments()


class ImageCaptionData(Dataset):
    def __init__(self, path='processed_caption.csv'):
        super().__init__()
        self.path = path
        self.data = pd.read_csv(path)
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        return transform(self.data.iloc[index]['file_name']), self.data.iloc[index]['caption_tokens']
        

def get_dataloader():
    dataloader = DataLoader(dataset=ImageCaptionData(), batch_size=args.batch_size, num_workers=args.num_workers, shuffle=True, pin_memory=args.pin_memory)
    return dataloader


def transform(img_path):
    transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((args.img_size, args.img_size)),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ])
    
    
    img = Image.open(img_path)
    transformed_img = transformer(img)
    
    return transformed_img

In [77]:
data = ImageCaptionData()

tensor(0.9925)